In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest, chi2

In [10]:
path = r'titanic.csv'

In [11]:
df = pd.read_csv(path)

In [12]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
df.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace = True)

In [14]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


# plan

## Missing value -- OHE--Scaling--Feature Selection---train using Decision tree

In [16]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'],
                                                 test_size = 0.2,
                                                 random_state = 42)

In [18]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [20]:
y_train.sample(5)

587    1
744    1
717    1
606    0
264    0
Name: Survived, dtype: int64

# Imputation transformer

In [23]:
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(),[2]), # index value of age as when the output is generated its in NUMPY ARRAY rather than a DATAFRAME
    ('impute_embarked',SimpleImputer(strategy = 'most_frequent'),[6]) #index of Embarked
], remainder = 'passthrough')

# One Hot Encoding

In [27]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse = False, handle_unknown = 'ignore'),[1,6])
], remainder ='passthrough')

# Scaling

In [29]:
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(),slice(0,10)) 
])
# Using min max scaler insted of standard scaler because plan is to use feature selection in ahed codes
#we could have skipped categorical columns here aswell
# 0 to 10 because (sex= 2) + (embarked = 3) new = 5 old =5  so 10 columns

# Feature Scaling

In [75]:
trf4 = SelectKBest(score_func = chi2, k=9) # K(column to use/ Imp Columns = 9 out of 10)

# Trainning the Model using Decision Tree

In [76]:
trf5 = DecisionTreeClassifier()

# Creating Pipeline

In [77]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

## Pipeline VS Make_Pipeline
***Pipeline requries naming of steps, make_pipeline does not.***

***(same applies to ColumnTransformer vs make_column_transformer)***

In [78]:
# Alternate Syntax using MAKE_PIPELINE
pipe_new = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [79]:
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=9,
                             score_func=<function chi2 at 0x000001F4DE6D6E80>)),
                ('trf5', DecisionTreeClassifier())])

In [80]:
pipe_new

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=9,
                             score_func=<function chi2 at 0x000001F4DE6D6E80>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [81]:
pipe.fit(x_train,y_train) # if only 3 steps were taken (impute, ohe, scaling) the pipe would use fit_transform or do ude fit and then in next line use transform

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=9,
                             score_func=<function chi2 at 0x000001F4DE6D6E80>)),
                ('trf5', DecisionTreeClassifier())])

# to display pipeline

In [82]:
from sklearn import set_config
set_config(display = 'diagram')

# explore the pipeline

In [83]:
pipe.steps

[('trf1',
  ColumnTransformer(remainder='passthrough',
                    transformers=[('impute_age', SimpleImputer(), [2]),
                                  ('impute_embarked',
                                   SimpleImputer(strategy='most_frequent'),
                                   [6])])),
 ('trf2',
  ColumnTransformer(remainder='passthrough',
                    transformers=[('ohe_sex_embarked',
                                   OneHotEncoder(handle_unknown='ignore',
                                                 sparse=False),
                                   [1, 6])])),
 ('trf3',
  ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))])),
 ('trf4', SelectKBest(k=9, score_func=<function chi2 at 0x000001F4DE6D6E80>)),
 ('trf5', DecisionTreeClassifier())]

In [84]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=9, score_func=<function chi2 at 0x000001F4DE6D6E80>),
 'trf5': DecisionTreeClassifier()}

In [85]:
pipe.classes_

array([0, 1], dtype=int64)

In [86]:
pipe.score

<bound method Pipeline.score of Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                          

In [87]:
pipe.named_steps #steps in the pipeline

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=9, score_func=<function chi2 at 0x000001F4DE6D6E80>),
 'trf5': DecisionTreeClassifier()}

**Lets say we need to find the mean value of simple imputer**

In [88]:
pipe.named_steps['trf1'] #SHOWS STEPS IN TRF1

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [89]:
pipe.named_steps['trf1'].transformers_ # list of transformers used here

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [90]:
pipe.named_steps['trf1'].transformers_[0] # slicing 1st transformer = tupple of 1st simple imputer

('impute_age', SimpleImputer(), [2])

In [91]:
pipe.named_steps['trf1'].transformers_[0][1] # getting the simple imputer

SimpleImputer()

In [92]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_ # ussing differentt attributes in simple imputer such as mean

array([29.49884615])

In [93]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

# Prediction

In [94]:
y_pred = pipe.predict(x_test)

In [95]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [96]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

# Cross Validation Using Pipeline

*Cross Validation using cross_val_score*

In [98]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train,cv = 20, scoring = 'accuracy').mean()

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. 

0.639126984126984

# HyperParameter Tunning

## Grid Search using Pipeline

In [102]:
params = {
    'trf5__max_depth' : [1,2,3,4,5,None] # trf5 is the name of our decison tree  and there is DOUBLE UNDRSCORE after that 
}

In [105]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv = 15, scoring = 'accuracy')
grid.fit(x_train, y_train)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. 

GridSearchCV(cv=15,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=9,
                                                    score_func=<function chi2 at 0x000001F4DE6D6E80>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [106]:
grid.best_score_

0.6391548463356974

In [107]:
grid.best_params_

{'trf5__max_depth': 1}

# Exporting the Pipeline

In [108]:
# Export
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))